In [11]:
from httpx import head
import pandas as pd
from regex import D
from sklearn.preprocessing import OneHotEncoder
df1 = pd.read_csv('../Data/OriginalData/silver_labels_Timestamp5_7.csv')#chat gpt labels old
df1["label"].replace("Mixed", "NO_STATEMENT", inplace=True)
statement_cats = (df1["label"].dropna().astype(str).unique().tolist())
print(statement_cats)#what does mixed stand for?
statement_enc = OneHotEncoder(categories=[statement_cats], sparse_output=False, handle_unknown="ignore")
statement_enc.fit(df1.loc[df1["label"].notna(), ["label"]])
onehot = statement_enc.transform(df1[["label"]])
conf = df1["confidence"].fillna(0).to_numpy().reshape(-1, 1)
df1["label_vec"] = (onehot * conf).tolist()
df1.drop(columns=["confidence"], inplace=True)
print(df1.head())
df2 = pd.read_csv('../Data/OriginalData/truth_labels_prefilterd_gpt5.csv')#chat gpt labels after sentiment classification 
df2["label_vec"] = df2[["TRUE", "FALSE", "NO_STATEMENT"]].values.tolist()
df2.rename(columns={"truth_label": "label"}, inplace=True)
df2.drop(columns=["Unnamed: 0", "TRUE", "FALSE", "NO_STATEMENT"], inplace=True)
print(df2.head())

df3 = pd.read_csv('../Data/OriginalData/truth_labels_fnn_gpt5.csv')
df3.rename(columns={"0": "id","1":"label"}, inplace=True)
style_cluster_cats = (df3["label"].dropna().astype(str).unique().tolist())
style_enc = OneHotEncoder(categories=[style_cluster_cats], sparse_output=False, handle_unknown="ignore")
style_enc.fit(df3.loc[df3["label"].notna(), ["label"]])
onehot = style_enc.transform(df3[["label"]])
df3["label_vec"] = onehot.tolist()


df6 = df2.merge(df1, on="id", how="outer",)
df6 = df6.merge(df3, on="id", how="outer")
df6["gpt_label"] = df6["label_x"].combine_first(df6["label_y"]).combine_first(df6["label"]) 
df6["gpt_label_vec"] = df6["label_vec_x"].combine_first(df6["label_vec_y"]).combine_first(df6["label_vec"])
df6 = df6.drop(columns=["label_x", "label_y", "label_vec_x", "label_vec_y"])

print(df6.columns)



df4 = pd.read_csv('../Data/OriginalData/truths.tsv', sep="\t")
df4 = df4[["id", "timestamp"]]
df5 = pd.read_csv('../Data/OriginalData/truthsocial_user_features.csv')


C:\Users\johan\AppData\Local\Temp\ipykernel_24208\636405232.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1["label"].replace("Mixed", "NO_STATEMENT", inplace=True)


['NO_STATEMENT', 'FALSE', 'TRUE', 'MIXED']
       id         label              label_vec
0  703265  NO_STATEMENT  [0.98, 0.0, 0.0, 0.0]
1  807621  NO_STATEMENT  [0.93, 0.0, 0.0, 0.0]
2  703273  NO_STATEMENT  [0.98, 0.0, 0.0, 0.0]
3  703274  NO_STATEMENT  [0.98, 0.0, 0.0, 0.0]
4  703305  NO_STATEMENT  [0.98, 0.0, 0.0, 0.0]
       id         label          label_vec
0  807621  NO_STATEMENT  [0.02, 0.08, 0.9]
1  703306  NO_STATEMENT  [0.05, 0.05, 0.9]
2  703308         FALSE  [0.05, 0.6, 0.35]
3  703394  NO_STATEMENT  [0.15, 0.15, 0.7]
4  703282  NO_STATEMENT  [0.02, 0.08, 0.9]
Index(['id', 'label', 'label_vec', 'gpt_label', 'gpt_label_vec'], dtype='object')


KeyboardInterrupt: 

In [ ]:
print(len(df1))
print(len(df2))
print(len(df3))
print(len(df6))

75131
25020
85300
164733


In [ ]:
from httpx import head
from polars import col
from sklearn.preprocessing import OneHotEncoder

df5 = df5.rename(columns={"timestamp": "timestamp_id"})
try:
    df5["bert_label_vec"] = df5[["TRUE", "FALSE", "NO_STMT"]].values.tolist()
    df5 = df5.drop(columns=["TRUTH_CLASS_x","NO_STMT","TRUE" ,"FALSE"])#bert classifier output  
except:
    pass
#try:
    #df2 = df2.drop(columns=["Unnamed: 0"])
#except:
#    pass
df3 = df3.rename(columns={"0": "id","1": "truth_label_fnn"})
#df1 = df1.rename(columns=lambda c: c if c == "id" or c.startswith("silver_label_") else f"silver_label_{c}")#these are chatgpt labels 
#try:
#    df2["bert_prediction"] = df2[["TRUE", "FALSE", "NO_STATEMENT"]].values.tolist()#these are also chat gpt labels 
#    df2.drop(columns=["TRUE", "FALSE", "NO_STATEMENT","truth_label"], inplace=True)
#except:
#    pass
print(len(df6))
df = df5.merge(df6, on="id", how="inner")
print(len(df))
df["bert_label_vec"] = df["bert_label_vec"].apply(
    lambda x: [0 if pd.isna(v) else v for v in x] if isinstance(x, list) else x
)
df=df.merge(df4, on="id", how="inner")
#df = df.merge(df2, on="id", how="inner")
#df = df.merge(df3, on="id", how="inner")
print(df.columns)
print(df6.columns)


print(df["author"].unique().shape)



80723
80727
Index(['id', 'timestamp_id', 'author', 'like_count', 'retruth_count',
       'reply_count', 'text', 'hate_pred', 'hate_prob', 'sentiment_id',
       'sentiment_conf', 'sentiment', 'statement_flag',
       'statement_probability', 'clusters', 'style_cluster',
       'activity_cluster', 'temporal_rhythm_cluster', 'topic_label',
       'bert_label_vec', 'gpt_label', 'gpt_label_vec', 'timestamp'],
      dtype='object')
Index(['id', 'gpt_label', 'gpt_label_vec'], dtype='object')
(1252,)


In [ ]:
print(df["activity_cluster"][:10])

0    5.0
1    2.0
2    3.0
3    3.0
4    3.0
5    3.0
6    3.0
7    3.0
8    3.0
9    3.0
Name: activity_cluster, dtype: float64


In [ ]:
# =========================
# Imports
# =========================
import torch
import torch.nn as nn
import numpy as np
from transformers import AutoTokenizer, AutoModel

# =========================
# CONFIG
# =========================
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
TEXT_DIM = 64          # node feature size for GNN
MAX_LEN =2048
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =========================
# BERT + FFN Encoder
# =========================
class BertFFNEncoder(nn.Module):
    def __init__(self, model_name, out_dim):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        
        # freeze BERT
        for p in self.bert.parameters():
            p.requires_grad = False
        
        hidden_dim = self.bert.config.hidden_size
        
        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            out = self.bert(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            cls_embedding = out.last_hidden_state[:, 0]  # [CLS]
        
        return self.ffn(cls_embedding)

# =========================
# Tokenizer & Model
# =========================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
encoder = BertFFNEncoder(MODEL_NAME, TEXT_DIM).to(DEVICE)
encoder.eval()

# =========================
# Encode df["text"]
# =========================
def encode_texts(texts, batch_size=32):
    features = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        enc = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt"
        )

        input_ids = enc["input_ids"].to(DEVICE)
        attention_mask = enc["attention_mask"].to(DEVICE)

        with torch.no_grad():
            vecs = encoder(input_ids, attention_mask)

        features.append(vecs.cpu().numpy())

    return np.vstack(features)

# =========================
# RUN ENCODING
# =========================
text_features = encode_texts(df["text"].tolist())

# =========================
# STORE AS NODE FEATURES
# =========================
#feature_df["text_feat"] = text_features.tolist()

# =========================
# SANITY CHECKS
# =========================
#print("Node feature dim:", len(feature_df["text_feat"].iloc[0]))
#print("Mean:", np.mean(feature_df["text_feat"].iloc[0]))
#print("Std:", np.std(feature_df["text_feat"].iloc[0]))


In [ ]:

#print(feature_df["text_feat"][:10])

known Columns: [id, timestamp, author, like_count, retruth_count, reply_count, text, hate_pred, hate_prob, sentiment_id, sentiment_conf, sentiment, style_cluster, activity_cluster, temporal_rhythm_cluster, topic_label, silver_label_label, silver_label_confidence, bert_prediction, truth_label_fnn]

Unknown Columns: [ statement_flag, statement_probability, TRUTH_CLASS_x, NO_STMT, TRUE, FALSE, clusters,]


In [ ]:
print(df.columns)

Index(['id', 'timestamp_id', 'author', 'like_count', 'retruth_count',
       'reply_count', 'text', 'hate_pred', 'hate_prob', 'sentiment_id',
       'sentiment_conf', 'sentiment', 'statement_flag',
       'statement_probability', 'clusters', 'style_cluster',
       'activity_cluster', 'temporal_rhythm_cluster', 'topic_label',
       'bert_label_vec', 'gpt_label', 'gpt_label_vec', 'timestamp'],
      dtype='object')


In [ ]:
from cv2 import norm
import numpy as np
from transformers import  AutoTokenizer
from yaml import Token

feature_df= df[["id","timestamp_id","timestamp","author", 'bert_label_vec',"gpt_label","gpt_label_vec"]].copy()
feature_df["text_feat"] = text_features.tolist()

temporal_rhythm_cluster_cats = (df["temporal_rhythm_cluster"].dropna().astype(str).unique().tolist())
style_cluster_cats = (df["style_cluster"].dropna().astype(str).unique().tolist())
activity_cluster_cats = (df["activity_cluster"].dropna().astype(str).unique().tolist())
topic_label_cats = (df["topic_label"].dropna().astype(str).unique().tolist())
sentiment_cats = (df["sentiment_id"].dropna().astype(str).unique().tolist())
sortcats = sorted(style_cluster_cats)
print(sortcats)


style_enc = OneHotEncoder(categories=[sortcats], sparse_output=False, handle_unknown="ignore")
style_enc.fit(df.loc[df["style_cluster"].notna(), ["style_cluster"]])
onehot = style_enc.transform(df[["style_cluster"]])
feature_df["style_cluster_cats"] = onehot.tolist()

topic_label_enc = OneHotEncoder(categories=[topic_label_cats], sparse_output=False, handle_unknown="ignore")
topic_label_enc.fit(df.loc[df["topic_label"].notna(), ["topic_label"]])
onehot = topic_label_enc.transform(df[["topic_label"]])
feature_df["topic_label_cats"] = onehot.tolist()

topic_label_enc = OneHotEncoder(categories=[topic_label_cats], sparse_output=False, handle_unknown="ignore")
topic_label_enc.fit(df.loc[df["topic_label"].notna(), ["topic_label"]])
onehot = topic_label_enc.transform(df[["topic_label"]])
feature_df["topic_label_cats"] = onehot.tolist()


sentiment_cats = sorted(sentiment_cats)
setiment_enc = OneHotEncoder(categories=[sentiment_cats], sparse_output=False, handle_unknown="ignore")
setiment_enc.fit(df.loc[df["sentiment_id"].notna(), ["sentiment_id"]])
onehot = setiment_enc.transform(df[["sentiment_id"]])
conf = df["sentiment_conf"].fillna(0).to_numpy().reshape(-1, 1)
feature_df["sentiment_vec"] = (onehot * conf).tolist()

activity_cluster_cats = sorted(activity_cluster_cats)
activity_cluster_enc = OneHotEncoder(categories=[activity_cluster_cats], sparse_output=False, handle_unknown="ignore")
activity_cluster_enc.fit(df.loc[df["activity_cluster"].notna(), ["activity_cluster"]])         
onehot = activity_cluster_enc.transform(df[["activity_cluster"]])
feature_df["activity_cluster_cats"] = onehot.tolist()

temporal_rhythm_cluster_cats = sorted(df["temporal_rhythm_cluster"].dropna().astype(str).unique().tolist())
temporal_rhythm_cluster_enc = OneHotEncoder(categories=[temporal_rhythm_cluster_cats], sparse_output=False, handle_unknown="ignore")
temporal_rhythm_cluster_enc.fit(df.loc[df["temporal_rhythm_cluster"].notna(), ["temporal_rhythm_cluster"]])
onehot = temporal_rhythm_cluster_enc.transform(df[["temporal_rhythm_cluster"]])
feature_df["temporal_rhythm_cluster_cats"] = onehot.tolist()


feature_df["hate_vec"] = df["hate_prob"]

statement_cats = (df["statement_flag"].dropna().astype(str).unique().tolist())
statement_enc = OneHotEncoder(categories=[statement_cats], sparse_output=False, handle_unknown="ignore")
statement_enc.fit(df.loc[df["statement_flag"].notna(), ["statement_flag"]])
onehot = statement_enc.transform(df[["statement_flag"]])
conf = df["statement_probability"].fillna(0).to_numpy().reshape(-1, 1)
feature_df["statement_vec"] = (onehot * conf).tolist()


x = np.log1p(df["like_count"])

mean = x.mean()
std = x.std()

feature_df["like_count_scaled"] = (x - mean) / std

x = np.log1p(df["retruth_count"])
mean = x.mean()
std = x.std()
feature_df["retruth_count_scaled"] = (x - mean) / std

x = np.log1p(df["reply_count"]) 
mean = x.mean()
std = x.std()
feature_df["reply_count_scaled"] = (x - mean) / std

#TODO: i need to deifine a dictionary 
#Tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#feature_df["text_tokenized"] = feature_df["id"].apply(lambda x: Tokenizer.encode(df.loc[df["id"] == x, "text"].values[0],))

#feature_df["hate_prob"] = df["hate_prob"]

import numpy as np
from collections import Counter

# parse clusters
parse = lambda x: [] if x is None or x != x else (
    x if isinstance(x, list)
    else [int(c) for c in str(x).strip("[]").split(",") if c.strip()]
)

clusters = df["clusters"].apply(parse)

# vocab
cats = sorted({c for row in clusters for c in row})
idx = {c: i for i, c in enumerate(cats)}

# encode + normalize
feature_df["cluster_vec"] = clusters.apply(
    lambda row: (
        lambda v: v / v.sum() if v.sum() > 0 else v
    )(np.bincount([idx[c] for c in row], minlength=len(cats)).astype(float))
).tolist()


['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0']


In [ ]:
feature_df.to_csv("../Data/ProcessedData/network_modeling_data.csv", index=False)

In [ ]:
from io import text_encoding
from regex import F


print(feature_df.columns)
print(feature_df.head())

Index(['id', 'timestamp_id', 'timestamp', 'author', 'bert_label_vec',
       'gpt_label', 'gpt_label_vec', 'text_feat', 'style_cluster_cats',
       'topic_label_cats', 'sentiment_vec', 'activity_cluster_cats',
       'temporal_rhythm_cluster_cats', 'hate_vec', 'statement_vec',
       'like_count_scaled', 'retruth_count_scaled', 'reply_count_scaled',
       'cluster_vec'],
      dtype='object')
       id  timestamp_id            timestamp  author  \
0  703265            13  2022-05-14 00:00:00    2247   
1  807621             6  2022-03-26 00:00:00   20054   
2  703273            25  2022-08-01 00:00:00   11225   
3  703274            25  2022-08-01 00:00:00   11225   
4  703305            24  2022-07-26 00:00:00   11225   

                           bert_label_vec     gpt_label  \
0                               [0, 0, 0]  NO_STATEMENT   
1  [0.020946145, 0.035621844, 0.94343203]  NO_STATEMENT   
2                               [0, 0, 0]  NO_STATEMENT   
3                            